# 🌲 Ray Tune - Random Forest Hyperparameter Tuning (CoverType Dataset)

## 📦 Imports

In [ ]:
import ray
from ray import tune
from sklearn.datasets import fetch_covtype
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


## 🚀 Connect to Ray Cluster

In [ ]:
ray.init(address="auto")  # Automatically connects to your Ray cluster


## 🎯 Define Training Function for Random Forest

In [ ]:
def train_model(config):
    cov = fetch_covtype()
    X, y = cov.data[:10000], cov.target[:10000]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = RandomForestClassifier(
        n_estimators=config["n_estimators"],
        max_depth=config["max_depth"],
        random_state=42
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    tune.report(accuracy=acc)


## 🔧 Run Hyperparameter Tuning

In [ ]:
search_space = {
    "n_estimators": tune.choice([50, 100, 200]),
    "max_depth": tune.choice([5, 10, 20, None]),
}

tuner = tune.Tuner(
    train_model,
    param_space=search_space
)

results = tuner.fit()


## ✅ Best Result

In [ ]:
best = results.get_best_result(metric="accuracy", mode="max")
print("Best config:", best.config)
print("Best accuracy:", best.metrics['accuracy'])
